# Gene Ontology (GO) Analysis

This notebook demonstrates how to work with Gene Ontology data in PerturbLab.

## Features
- Download and load GO ontology files
- Compute gene-gene similarity from GO annotations
- Build perturbation graphs for GEARS
- Query GO terms and gene annotations


In [ ]:
from perturblab.data.resources import load_dataset
from perturblab.utils import read_obo
from perturblab.types import GeneVocab

# Download GO ontology file
go_path = load_dataset('go/go_basic')
print(f"GO file: {go_path}")

# Read GO ontology
ontology = read_obo(go_path)
print(f"\nLoaded {len(ontology)} GO terms")
print(f"Example term: {list(ontology.keys())[0]}")
print(f"Term info: {ontology[list(ontology.keys())[0]]}")


## Compute Gene Similarity from GO Annotations


In [ ]:
from perturblab.tools import compute_gene_similarity_from_go

# Example gene-to-GO mapping
gene_to_go = {
    'TP53': {'GO:0006915', 'GO:0006974', 'GO:0006281'},
    'BRCA1': {'GO:0006281', 'GO:0006974', 'GO:0000724'},
    'KRAS': {'GO:0007165', 'GO:0007264', 'GO:0008284'},
    'MYC': {'GO:0008284', 'GO:0006355', 'GO:0000122'},
}

# Compute similarity graph
similarity_graph = compute_gene_similarity_from_go(
    gene_to_go,
    similarity='jaccard',
    threshold=0.1,
    show_progress=True
)

print(f"Similarity graph edges: {len(similarity_graph)}")
print(f"\nFirst 5 edges:")
print(similarity_graph.head())


## Build Perturbation Graph for GEARS


In [ ]:
from perturblab.methods.gears import build_perturbation_graph

# Create gene vocabulary
gene_vocab = GeneVocab(['TP53', 'BRCA1', 'KRAS', 'MYC', 'EGFR', 'BRCA2'])

# Build perturbation graph (automatically downloads GO annotations if needed)
pert_graph = build_perturbation_graph(
    gene_vocab,
    similarity='jaccard',
    threshold=0.1,
    num_workers=1,
    show_progress=True
)

print(f"Graph nodes: {pert_graph.n_nodes}")
print(f"Graph edges: {pert_graph.n_unique_edges}")
print(f"\nNeighbors of TP53: {pert_graph.neighbors('TP53')}")
print(f"Edge weights: {pert_graph.get_weights('TP53')}")
